# Imports

In [1]:
import ta
import numpy as np
from ta import add_all_ta_features
import pandas as pd
from ta.volatility import AverageTrueRange
import upstox_client 
from __future__ import print_function
import time
from upstox_client.rest import ApiException
from pprint import pprint

# Main code

In [2]:
# Function that returns the Heikin Ashi version of the data

def heikinaashi(df):
    new_df = []
    open = (df.iloc[0]['Open']+df.iloc[0]['Close'])/2
    close = (df.iloc[0]['Open']+df.iloc[0]['Close']+df.iloc[0]['High']+df.iloc[0]['Low'])/4
    high = max(df.iloc[0]['High'], open, close)
    low = max(df.iloc[0]['Low'], open, close)

    new_df.append({
        'Date':df.iloc[0]['Date'],
        'Open':open,
        'High':high,
        'Low':low,
        'Close':close      
    })
    
    for i in range(1, len(df)):
        open = (df.iloc[i-1]['Open']+df.iloc[i-1]['Close'])/2
        close = (df.iloc[i]['Open']+df.iloc[i]['Close']+df.iloc[i]['High']+df.iloc[i]['Low'])/4
        high = max(df.iloc[i]['High'], open, close)
        low = max(df.iloc[i]['Low'], open, close)
        new_df.append({
            'Date':df.iloc[i]['Date'],
            'Open':open,
            'High':high,
            'Low':low,
            'Close':close        
        })

    new_df= pd.DataFrame(new_df, index=df.index)
    return new_df        

In [3]:
# Function that returns values for ema 1, 2, and 3

def ema_entries(df, ema1, ema2, ema3):
    df['EMA'+str(ema1)] = df['Close'].ewm(span=ema1, adjust=False).mean()
    df['EMA'+str(ema2)] = df['Close'].ewm(span=ema2, adjust=False).mean()
    df['EMA'+str(ema3)] = df['Close'].ewm(span=ema3, adjust=False).mean()
    return df

In [4]:
def ema_satisfy(ema1, ema2, ema3):
    if((ema1>ema2) & (ema2>ema3)):
        return True
    return False

In [5]:
# Function that adds a supertrend and direction column

def superTrend(df, length, multiplier):
    atr = AverageTrueRange(high = df['High'], low = df['Low'], close = df['Close'], window = length).average_true_range()
    hl2 = (df['High'] + df['Low']) / 2
    basicUpperBand = hl2 + (multiplier * atr)
    basicLowerBand = hl2 - (multiplier * atr)

    upperBand = [np.nan] * len(df)
    lowerBand = [np.nan] * len(df)
    superTrend = [np.nan] * len(df)
    trendDirection = [False] * len(df)


    for i in range(length, len(df)):
        prev_upperBand = upperBand[i-1] if i>length else np.nan
        prev_close = df.iloc[i-1]['Close']
        prev_lowerBand = lowerBand[i-1] if i>length else np.nan
        prev_superTrend = superTrend[i-1]

        if(np.isnan(prev_upperBand) or (basicUpperBand[i] < prev_upperBand) or (prev_close > prev_upperBand) ):
            upperBand[i] = basicUpperBand[i]
        else:
            upperBand[i] = prev_upperBand

        if(np.isnan(prev_lowerBand) or basicLowerBand[i] > prev_lowerBand or prev_close < prev_lowerBand ):
            lowerBand[i] = basicLowerBand[i]
        else:
            lowerBand[i] = prev_lowerBand

        if np.isnan(prev_superTrend):
            trendDirection[i] = False      
        elif prev_superTrend == prev_upperBand:
            trendDirection[i] = True if (df.iloc[i]['Close'] > upperBand[i]) else False
        else:
            trendDirection[i] = False if (df.iloc[i]['Close'] < lowerBand[i]) else True

        superTrend[i] = lowerBand[i] if trendDirection[i] else upperBand[i]

    df['SuperTrend'] = superTrend
    df['SuperTrendDirection'] = trendDirection
    return df        

In [6]:
# Function that returns confirmation dates

def confirmation(df, df_entries, confirmation_locs):
    confirmation_points = []
    for i in confirmation_locs:
        while i<len(df):
            if df.iloc[i]['Open'] < df.iloc[i]['Close']:
                i += 1
            else:
                break
                
        while i<len(df):
            if df.iloc[i]['Open'] > df.iloc[i]['Close']:
                i += 1
            else:
                break
        if i<len(df):
            confirmation_points.append(df.iloc[i]['Date'])
        else:
            confirmation_points.append('None')

        
    df_entries['ConfirmationCandle'] = confirmation_points
    return df_entries

In [7]:
# Function that returns list of entry dates along with confirmation points

def entry(instrument_key, ema1, ema2, ema3, length, multiplier, from_date, to_date):

    api_instance = upstox_client.HistoryV3Api()
    unit = 'days'
    interval = 1 
    
    try:
        api_response = api_instance.get_historical_candle_data1(instrument_key, unit, interval, to_date, from_date)
    except ApiException as e:
        print("Exception when calling HistoryV3Api->get_historical_candle_data1: %s\n" % e)

    api_response.data.candles.reverse()
    df = pd.DataFrame(data = api_response.data.candles, columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends'])    
    df = df.drop(['Volume', 'Dividends'], axis=1)
    df = ema_entries(df, ema1, ema2, ema3)
    df = superTrend(df, length, multiplier)
    df.to_csv("final.csv")
    entries = []
    confirmation_locs = []
    for i in range(1, len(df)):
        if(ema_satisfy(df.iloc[i]['EMA9'], df.iloc[i]['EMA30'], df.iloc[i]['EMA100']) and not ema_satisfy(df.iloc[i-1]['EMA9'], df.iloc[i-1]['EMA30'], df.iloc[i-1]['EMA100']) and df.iloc[i]['SuperTrendDirection']==True):
            entries.append(df.iloc[i])
            confirmation_locs.append(i)
            
    df_entries = pd.DataFrame(entries)
    df_entries.to_csv("entrypoints.csv")

    df_confirmation = confirmation(df, df_entries, confirmation_locs)
    df_entries.to_csv("confirmationpoints.csv")

In [8]:
entry("NSE_EQ|INE002A01018", 9, 30, 100, 10, 3, '2023-08-01', '2025-07-07')

# Indices

In [ ]:
indices = {
    'RELIANCE':'NSE_EQ|INE002A01018',
    'HCLTECH':'NSE_EQ|INE860A01027',
}

In [30]:
url = 'https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz'
df = pd.read_json(url, compression='gzip')
result = df[(df['trading_symbol'] == 'RELIANCE') & (df['segment'] == 'NSE_EQ')]
print(result[['instrument_key', 'isin']])

            instrument_key          isin
45359  NSE_EQ|INE002A01018  INE002A01018
